<a href="https://colab.research.google.com/github/kkipngenokoech/agents/blob/main/flight_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building an Intelligent Travel Agent with LangChain

This tutorial walks step‑by‑step through a **LangChain‑based agent**.

* Utility functions
* Tools (functions the agent can call)
* Agent reasoning and decision‑making
* A simple interactive experience

The goal is **conceptual clarity**, not complexity.

This tutorial is presented as a standalone introduction to building intelligent agents using LangChain.

## Why LangChain for Agentic AI?

LangChain is designed to support the construction of **agentic systems**—systems that can reason, interact with users, and take actions using tools.

It provides structured abstractions for:

* Defining tools that agents can invoke
* Managing conversational context and intermediate reasoning
* Coordinating multi-step decision-making loops
* Executing tool calls automatically when sufficient information is available

By handling these concerns at the framework level, LangChain allows developers to focus on **agent behavior and logic** rather than low-level orchestration.

LangChain also supports easy extensibility, enabling agents to be augmented with memory, retrieval mechanisms, and more complex workflows as needed.

## Installing Required Libraries

In [ ]:
! pip install -q google-search-results
!pip install -q langchain-openai langchain


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 3.3 MB/s eta 0:00:00


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


In [ ]:
import getpass
import json
from typing import Dict, Optional
import ipywidgets as widgets
from IPython.display import display

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

## Setting Up the OpenAI API Key

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
SEARCHAPI_KEY = userdata.get('SEARCHAPI_KEY')
# AVIATIONSTACK_KEY = "ea5bfcb3808cfe25df1a4a26b7fc41e7"

## Define the flight booking tool

In [ ]:
import requests
import json
from typing import Optional
import random

@tool
def search_flights(
    from_city: str,
    to_city: str,
    travel_date: str,
    return_date: Optional[str] = None,
    passengers: int = 1,
    cabin_class: str = "economy"
) -> str:
    """
    Search for available flights using SearchAPI's Google Flights integration.
    Returns real flight data or simulated data as fallback.
    """
    api_key = SEARCHAPI_KEY  # Your SearchAPI key

    if not api_key:
        return "❌ API key not set. Using simulated data."

    # Convert to IATA codes (simple approach - in production use proper airport database)
    from_iata = from_city[:3].upper() if len(from_city) >= 3 else from_city.upper()
    to_iata = to_city[:3].upper() if len(to_city) >= 3 else to_city.upper()

    print(f"🔍 Searching: {from_iata} → {to_iata} on {travel_date}")

    try:
        # SearchAPI Google Flights endpoint
        url = "https://www.searchapi.io/api/v1/search"

        params = {
            "engine": "google_flights",
            "hl": "en",
            "api_key": api_key
        }

        # Set trip type
        if return_date:
            params["type"] = "2"  # Round trip
            params["outbound_date"] = travel_date
            params["return_date"] = return_date
        else:
            params["type"] = "1"  # One way
            params["outbound_date"] = travel_date

        # Add departure and arrival
        params["departure_id"] = from_iata
        params["arrival_id"] = to_iata

        # Add number of adults (passengers)
        params["adults"] = str(passengers)

        # Add cabin class if specified
        class_mapping = {
            "economy": "econ",
            "business": "bus",
            "first": "first"
        }
        if cabin_class in class_mapping:
            params["travel_class"] = class_mapping[cabin_class]

        print(f"📊 API params: {params}")

        response = requests.get(url, params=params, timeout=30)
        print(f"📡 Status Code: {response.status_code}")

        if response.status_code == 200:
            data = response.json()

            # Check for API errors first
            if "error" in data:
                print(f"⚠️ API Error: {data['error']}")
                return generate_simulated_flights(from_city, to_city, travel_date, return_date, passengers, cabin_class)

            # Check if we have flight results
            if "best_flights" in data and data["best_flights"]:
                # Process real flight data from SearchAPI
                flights = []

                # Take up to 5 best flights
                for flight_data in data["best_flights"][:5]:
                    try:
                        # Get airline and flight number
                        first_flight = flight_data.get("flights", [{}])[0]
                        airline = first_flight.get("airline", "Unknown")
                        flight_num = first_flight.get("flight_number", "N/A")

                        # Get departure and arrival times
                        departure = flight_data.get("departure_airport", {})
                        arrival = flight_data.get("arrival_airport", {})

                        dep_time = departure.get("time", "")
                        arr_time = arrival.get("time", "")

                        # Get duration
                        duration = flight_data.get("duration", 0)
                        hours = duration // 60
                        minutes = duration % 60
                        duration_str = f"{hours}h{minutes}m"

                        # Get price - SearchAPI returns price string like "$200"
                        price_str = flight_data.get("price", "$0")

                        # Convert to numeric and adjust for class
                        try:
                            price = float(price_str.replace("$", "").replace(",", ""))
                            if cabin_class == "business":
                                price *= 1.8
                            elif cabin_class == "first":
                                price *= 2.5
                            price *= passengers
                        except:
                            # Fallback pricing
                            base_price = random.randint(200, 600)
                            if cabin_class == "business":
                                base_price *= 2.5
                            elif cabin_class == "first":
                                base_price *= 4
                            price = base_price * passengers

                        flight_info = {
                            "airline": airline,
                            "flight": flight_num,
                            "departure": dep_time,
                            "arrival": arr_time,
                            "duration": duration_str,
                            "price": f"${price:.2f}"
                        }
                        flights.append(flight_info)

                    except Exception as e:
                        print(f"⚠️ Error processing flight data: {e}")
                        continue

                if flights:
                    # Format results
                    trip_type = "ROUND TRIP" if return_date else "ONE-WAY"
                    result = f"✈️ REAL FLIGHTS: {from_iata} → {to_iata} ({trip_type})\n"
                    result += f"Date: {travel_date}"
                    if return_date:
                        result += f" | Return: {return_date}"
                    result += f" | Passengers: {passengers} | Class: {cabin_class}\n\n"

                    for i, f in enumerate(flights, 1):
                        result += f"{i}. {f['airline']} Flight {f['flight']}\n"
                        result += f"   🛫 Depart: {f['departure']} → 🛬 Arrive: {f['arrival']}\n"
                        result += f"   ⏱️ Duration: {f['duration']} | 💰 Price: {f['price']}\n\n"

                    return result
            else:
                print("📭 No flight results found in API response")
                if "search_metadata" in data:
                    print(f"📊 Search metadata: {data['search_metadata']}")

        # Fallback if no real flights found
        return generate_simulated_flights(from_city, to_city, travel_date, return_date, passengers, cabin_class)

    except requests.exceptions.Timeout:
        print("⏱️ API request timed out")
        return generate_simulated_flights(from_city, to_city, travel_date, return_date, passengers, cabin_class)
    except requests.exceptions.RequestException as e:
        print(f"🌐 Network error: {e}")
        return generate_simulated_flights(from_city, to_city, travel_date, return_date, passengers, cabin_class)
    except Exception as e:
        print(f"⚠️ Unexpected error: {e}")
        return generate_simulated_flights(from_city, to_city, travel_date, return_date, passengers, cabin_class)

def generate_simulated_flights(from_city, to_city, date, return_date, passengers, cabin_class):
    """Generate simulated flight data"""
    airlines = ["Delta", "American", "United", "JetBlue", "Southwest", "Spirit", "Frontier", "Alaska"]

    flights = []

    for i in range(5):
        airline = random.choice(airlines)
        base_price = random.randint(180, 450)

        # Adjust price based on cabin class
        if cabin_class == "business":
            base_price *= 2.5
        elif cabin_class == "first":
            base_price *= 4

        # Adjust for passengers
        total_price = base_price * passengers

        # Generate random times
        dep_hour = random.randint(6, 12)
        dep_minute = random.choice(['00', '15', '30', '45'])
        arr_hour = dep_hour + random.randint(1, 6)
        arr_minute = random.choice(['00', '20', '40'])

        # Duration calculation
        duration_hours = arr_hour - dep_hour
        if int(arr_minute) < int(dep_minute):
            duration_hours -= 1
            duration_minutes = 60 + int(arr_minute) - int(dep_minute)
        else:
            duration_minutes = int(arr_minute) - int(dep_minute)

        flight_info = {
            "airline": airline,
            "flight": f"{airline[:2]}{random.randint(100, 999)}",
            "departure": f"{dep_hour}:{dep_minute}",
            "arrival": f"{arr_hour}:{arr_minute}",
            "duration": f"{duration_hours}h{duration_minutes}m",
            "price": f"${total_price:.2f}"
        }
        flights.append(flight_info)

    # Format results
    trip_type = "ROUND TRIP" if return_date else "ONE-WAY"
    result = f"✈️ SIMULATED FLIGHTS: {from_city} → {to_city} ({trip_type})\n"
    result += f"Date: {date}"
    if return_date:
        result += f" | Return: {return_date}"
    result += f" | Passengers: {passengers} | Class: {cabin_class}\n\n"

    for i, f in enumerate(flights, 1):
        result += f"{i}. {f['airline']} {f['flight']}\n"
        result += f"   🛫 {f['departure']} → 🛬 {f['arrival']} ({f['duration']})\n"
        result += f"   💰 {f['price']}\n\n"

    result += "📝 Note: These are simulated flights. Real API returned no results.\n"
    result += "✅ Ready to book? Provide passenger name and preferred flight number."

    return result

In [ ]:
# tool
# def book_flight(
#     passenger_name: str,
#     from_city: str,
#     to_city: str,
#     travel_date: str
# ) -> str:
#     """
#     Book a flight for the customer once all required details are known.

#     Args:
#         passenger_name: The passenger's legal name
#         from_city: The departure city
#         to_city: The arrival city
#         travel_date: The date of travel

#     Returns:
#         A confirmation message with booking details
#     """
#     # In a real application, this would interact with a booking API
#     confirmation = {
#         "status": "success",
#         "message": f"A {travel_date} flight has been booked from {from_city} to {to_city} for {passenger_name}",
#         "booking_reference": f"BK{hash(passenger_name + travel_date) % 100000:05d}"
#     }
#     return json.dumps(confirmation)

In [ ]:
import time
import random
from typing import Optional
from datetime import datetime, timedelta

@tool
def book_flight(
    passenger_name: str,
    from_city: str,
    to_city: str,
    travel_date: str,
    flight_number: Optional[str] = None,
    airline: Optional[str] = None,
    seat_preference: Optional[str] = None,
    passengers: int = 1,
    cabin_class: str = "economy"
) -> str:
    """
    Book a flight for the customer once all required details are known.
    Integrates with search_flights to find available options before booking.

    Args:
        passenger_name: The passenger's legal name
        from_city: The departure city
        to_city: The arrival city
        travel_date: The date of travel
        flight_number: Optional specific flight number to book
        airline: Optional specific airline to book with
        seat_preference: Optional seat preference (window, aisle, middle)
        passengers: Number of passengers (default: 1)
        cabin_class: Cabin class (economy, business, first)

    Returns:
        A confirmation message with booking details
    """
    print(f"✈️ Processing booking request for {passenger_name}...")
    print(f"   Route: {from_city} → {to_city}")
    print(f"   Date: {travel_date}")

    # First, search for flights to get options
    print(f"🔍 Searching for available flights...")

    try:
        # Call the search_flights function
        search_result = search_flights(
            from_city=from_city,
            to_city=to_city,
            travel_date=travel_date,
            passengers=passengers,
            cabin_class=cabin_class
        )

        # Check if we have any flight options
        if ("no flights" in search_result.lower() or
            "❌" in search_result or
            "simulated" in search_result.lower() or
            "cannot book" in search_result.lower()):
            print("⚠️ No suitable flights found, creating custom booking...")

            # Generate custom flight if none found
            if not flight_number:
                flight_number = f"{random.choice(['AA', 'DL', 'UA', 'WN'])}{random.randint(100, 999)}"
            if not airline:
                airline = random.choice(["American Airlines", "Delta", "United", "Southwest"])

        else:
            # Extract flight options from search result
            print("✅ Flight options found, proceeding with booking...")

            # If user didn't specify flight number or airline, use first available
            if not flight_number and not airline and "1." in search_result:
                # Try to extract first flight details
                lines = search_result.split("\n")
                for i, line in enumerate(lines):
                    if line.strip().startswith("1."):
                        # Get the flight info line
                        flight_line = line.strip()[2:].strip()
                        parts = flight_line.split()
                        if len(parts) > 1:
                            if not airline:
                                # Try to extract airline name
                                possible_airlines = ["Delta", "American", "United", "JetBlue",
                                                   "Southwest", "Spirit", "Frontier", "Alaska"]
                                for pa in possible_airlines:
                                    if pa in flight_line:
                                        airline = pa
                                        break
                            if not flight_number:
                                # Look for flight number pattern
                                for part in parts:
                                    if any(airline_code in part for airline_code in ["AA", "DL", "UA", "WN", "B6", "NK", "F9", "AS"]):
                                        flight_number = part
                                        break
                        break

    except Exception as e:
        print(f"⚠️ Error during flight search: {e}")
        # Fallback flight details
        if not flight_number:
            flight_number = f"AA{random.randint(100, 999)}"
        if not airline:
            airline = "American Airlines"

    # Generate final flight details
    if not flight_number:
        flight_number = f"FL{random.randint(1000, 9999)}"
    if not airline:
        airline = random.choice(["American Airlines", "Delta Air Lines", "United Airlines", "Southwest Airlines"])

    # Generate booking details
    print("📋 Generating booking details...")
    time.sleep(1)  # Simulate processing time

    # Generate booking reference with timestamp
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    booking_ref = f"BK{timestamp[-8:]}{abs(hash(passenger_name)) % 10000:04d}"

    # Generate seat assignment
    if seat_preference:
        seat_pref = seat_preference.lower()
        if seat_pref in ["window", "w"]:
            seat_row = random.randint(5, 30)
            seat_side = random.choice(["A", "F"])
            assigned_seat = f"{seat_row}{seat_side}"
        elif seat_pref in ["aisle", "a"]:
            seat_row = random.randint(5, 30)
            seat_side = random.choice(["C", "D"])
            assigned_seat = f"{seat_row}{seat_side}"
        elif seat_pref in ["middle", "m"]:
            seat_row = random.randint(5, 30)
            seat_side = "B" if random.choice([True, False]) else "E"
            assigned_seat = f"{seat_row}{seat_side}"
        else:
            seat_row = random.randint(5, 30)
            seat_side = random.choice(["A", "B", "C", "D", "E", "F"])
            assigned_seat = f"{seat_row}{seat_side}"
    else:
        seat_row = random.randint(5, 30)
        seat_side = random.choice(["A", "B", "C", "D", "E", "F"])
        assigned_seat = f"{seat_row}{seat_side}"

    # Generate gate and terminal
    gate = random.choice(["A", "B", "C", "D"]) + str(random.randint(1, 60))
    terminal = random.randint(1, 8)

    # Generate departure time
    dep_time_obj = datetime.strptime(travel_date, "%Y-%m-%d") if "-" in travel_date else datetime.strptime(travel_date, "%m/%d/%Y")
    dep_time_obj = dep_time_obj.replace(hour=random.randint(6, 20), minute=random.choice([0, 15, 30, 45]))
    departure_time = dep_time_obj.strftime("%I:%M %p")

    # Generate arrival time (1-6 hours later)
    arrival_time = (dep_time_obj + timedelta(hours=random.randint(1, 6), minutes=random.randint(0, 45))).strftime("%I:%M %p")

    # Generate IATA codes
    from_iata = from_city[:3].upper() if len(from_city) >= 3 else from_city.upper()
    to_iata = to_city[:3].upper() if len(to_city) >= 3 else to_city.upper()

    # Generate price based on cabin class and passengers
    base_price = random.randint(200, 600)
    if cabin_class == "business":
        base_price *= 2.5
    elif cabin_class == "first":
        base_price *= 4
    total_price = base_price * passengers

    # Create confirmation
    print("📧 Generating booking confirmation...")
    time.sleep(0.5)

    confirmation = f"""
    {'='*60}
    ✅ FLIGHT BOOKING CONFIRMED! ✅
    {'='*60}

    📋 PASSENGER & BOOKING DETAILS:
    {'─' * 40}
    👤 Passenger: {passenger_name}
    👥 Number of Passengers: {passengers}
    🔐 Booking Reference: {booking_ref}
    🎫 E-Ticket Number: ETKT-{booking_ref}
    📅 Booking Date: {datetime.now().strftime('%Y-%m-%d %I:%M %p')}

    ✈️ FLIGHT DETAILS:
    {'─' * 40}
    🛫 Departure: {from_city} ({from_iata})
    🛬 Arrival: {to_city} ({to_iata})
    📅 Travel Date: {travel_date}
    🕐 Departure Time: {departure_time}
    🕐 Arrival Time: {arrival_time}
    🏢 Airline: {airline}
    🔢 Flight Number: {flight_number}
    💺 Cabin Class: {cabin_class.title()}
    🪑 Assigned Seat: {assigned_seat}
    {f'✨ Seat Preference: {seat_preference.title()}' if seat_preference else ''}

    🏛️ AIRPORT INFORMATION:
    {'─' * 40}
    📍 Terminal: {terminal}
    🚪 Gate: {gate}
    ⏰ Check-in Opens: 24 hours before departure
    ⏰ Recommended Arrival: 2 hours before departure

    💰 PAYMENT DETAILS:
    {'─' * 40}
    Total Amount: ${total_price:.2f}
    💳 Payment Status: CONFIRMED
    📧 Receipt: Sent to registered email

    ⚠️ IMPORTANT REMINDERS:
    {'─' * 40}
    • Check in online 24 hours before departure
    • Have passport/ID ready at the airport
    • Baggage allowance: 1 carry-on + 1 personal item
    • Changes/cancellations may incur fees

    {'='*60}
    🎉 Thank you for choosing our flight service! 🎉
    {'='*60}

    Need assistance? Contact us at support@flightbookings.com
    """

    # Log booking
    print(f"📝 Booking logged: {booking_ref} for {passenger_name}")
    print(f"💰 Total charged: ${total_price:.2f}")

    return confirmation

Key idea:

* The **docstring** tells the LLM *when* to use this tool
* LangChain handles argument extraction automatically

## Creating the Language Model

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    openai_api_key=OPENAI_API_KEY
)

Why temperature = 0?

* Encourages deterministic behavior
* Better for task-oriented agents

## Designing the Agent Prompt
The prompt defines **agent behavior**, not answers.

In [ ]:
SYSTEM_PROMPT = """You are a helpful travel agent assistant.

        Your role is to help customers book flights by gathering the following information:
        - Passenger's full legal name
        - Departure city
        - Arrival city
        - Travel date

        IMPORTANT GUIDELINES:
- Always ask for missing information before using tools
- When searching flights, ask for: departure city, arrival city, travel date
- When booking flights, you need: passenger name, departure city, arrival city, travel date
- Be conversational and remember previous context
- After searching flights, suggest booking options
- Provide clear, helpful responses

        TOOLS AVAILABLE:
1. search_flights - Use when user wants to find REAL flight options (connects to Google Flights)
2. book_flight - Use when user wants to actually book a flight



        If you don't have all the required information, ask polite follow-up questions.
        When you have all the details, use the book_flight tool to complete the booking.

        Be friendly, professional, and efficient in your interactions. Always show your thinking process and explain what you're doing."""

## Creating the Agent

In [ ]:
import json
from typing import List, Dict, Any, Optional
import traceback

class SimpleTravelAgent:
    def __init__(self, llm, tools: List, system_prompt: str):
        self.llm = llm
        self.tools = {tool.name: tool for tool in tools}
        self.system_prompt = system_prompt
        self.conversation_history = []
        self.max_history = 12  # Limit conversation history to manage tokens

    def add_to_history(self, role: str, content: str):
        """Add message to conversation history"""
        self.conversation_history.append({"role": role, "content": content})

        # Keep history within limits
        if len(self.conversation_history) > self.max_history * 2:
            # Keep system prompt + recent messages
            self.conversation_history = self.conversation_history[-self.max_history:]

    def clear_history(self):
        """Clear conversation history"""
        self.conversation_history = []
        print("🗑️ Conversation history cleared.")

    def get_tool_descriptions(self) -> str:
        """Generate descriptions of available tools for the LLM"""
        tool_descs = []
        for tool_name, tool in self.tools.items():
            desc = f"- {tool_name}: {tool.description}"
            tool_descs.append(desc)
        return "\n".join(tool_descs)

    def extract_tool_calls(self, response) -> List[Dict]:
        """Extract tool calls from LLM response, handling different response formats"""
        tool_calls = []

        # Check for tool_calls attribute (common in newer LLMs)
        if hasattr(response, 'tool_calls') and response.tool_calls:
            return response.tool_calls

        # Check for additional_kwargs (common in LangChain/Anthropic format)
        elif hasattr(response, 'additional_kwargs') and response.additional_kwargs:
            tool_calls_data = response.additional_kwargs.get('tool_calls', [])
            if tool_calls_data:
                return tool_calls_data

        # Check if response content contains tool call markers
        elif hasattr(response, 'content'):
            content = response.content
            # Look for JSON-like tool calls in content
            import re
            json_pattern = r'\{.*?"name":.*?"args":.*?\}'
            matches = re.findall(json_pattern, content, re.DOTALL)
            for match in matches:
                try:
                    tool_call = json.loads(match)
                    if 'name' in tool_call and 'args' in tool_call:
                        tool_calls.append({
                            'name': tool_call['name'],
                            'args': tool_call['args'],
                            'id': f"call_{len(tool_calls)}"
                        })
                except:
                    pass

        return tool_calls

    def format_tool_result(self, tool_name: str, result: Any, max_length: int = 500) -> str:
        """Format tool result for LLM consumption"""
        if isinstance(result, str):
            # Truncate very long results
            if len(result) > max_length:
                return result[:max_length] + "...[truncated]"
            return result
        elif isinstance(result, (dict, list)):
            try:
                formatted = json.dumps(result, indent=2)
                if len(formatted) > max_length:
                    return formatted[:max_length] + "...[truncated]"
                return formatted
            except:
                return str(result)[:max_length]
        else:
            return str(result)[:max_length]

    def process_message(self, user_input: str) -> str:
        """Process a user message and return agent's response"""
        try:
            # Add user message to history
            self.add_to_history("user", user_input)

            # Prepare enhanced system prompt with tool descriptions
            enhanced_system_prompt = f"""{self.system_prompt}

AVAILABLE TOOLS:
{self.get_tool_descriptions()}

INSTRUCTIONS:
1. First, understand the user's request
2. If flight search is needed, use search_flights tool
3. If booking is requested, use book_flight tool with appropriate details
4. Always ask for missing information before booking
5. Provide clear, helpful responses

CONVERSATION HISTORY (last {self.max_history} messages):
"""

            # Prepare messages for LLM
            messages = [{"role": "system", "content": enhanced_system_prompt}]

            # Add conversation history
            for msg in self.conversation_history[-self.max_history:]:
                messages.append(msg)

            # Get response from LLM
            print(f"🤖 Processing request: {user_input[:50]}...")
            response = self.llm.invoke(messages)

            # Extract tool calls
            tool_calls = self.extract_tool_calls(response)

            if tool_calls:
                tool_results = []
                for tool_call in tool_calls:
                    tool_name = tool_call.get('name')
                    tool_args = tool_call.get('args', {})

                    if not tool_name:
                        print("⚠️ Warning: Tool call missing name")
                        continue

                    if tool_name in self.tools:
                        # Display tool usage
                        print(f"🛠️ Using tool: {tool_name}")
                        print(f"📋 Tool input: {tool_args}")

                        try:
                            # Execute tool
                            if isinstance(tool_args, dict):
                                tool_result = self.tools[tool_name].invoke(tool_args)
                            else:
                                # Try to convert string args to dict
                                try:
                                    if isinstance(tool_args, str):
                                        tool_args = json.loads(tool_args)
                                    tool_result = self.tools[tool_name].invoke(tool_args)
                                except:
                                    tool_result = f"Error: Could not parse tool arguments: {tool_args}"

                            # Format and display result
                            formatted_result = self.format_tool_result(tool_name, tool_result)
                            print(f"✅ Tool '{tool_name}' completed successfully")

                            # Add tool call and result to history
                            self.add_to_history("assistant", f"I used the {tool_name} tool")
                            tool_results.append({
                                "tool_call_id": tool_call.get('id', f"call_{len(tool_results)}"),
                                "name": tool_name,
                                "result": formatted_result
                            })

                        except Exception as e:
                            error_msg = f"Error executing tool {tool_name}: {str(e)}"
                            print(f"❌ {error_msg}")
                            tool_results.append({
                                "tool_call_id": tool_call.get('id', f"call_{len(tool_results)}"),
                                "name": tool_name,
                                "result": error_msg
                            })
                    else:
                        error_msg = f"Tool '{tool_name}' not found. Available tools: {list(self.tools.keys())}"
                        print(f"❌ {error_msg}")
                        tool_results.append({
                            "tool_call_id": tool_call.get('id', f"call_{len(tool_results)}"),
                            "name": tool_name,
                            "result": error_msg
                        })

                # If we used tools, get final response from LLM with tool results
                if tool_results:
                    # Add the assistant's tool-calling message
                    messages.append({
                        "role": "assistant",
                        "content": response.content if hasattr(response, 'content') else "",
                        "tool_calls": tool_calls
                    })

                    # Add tool results as tool messages
                    for result in tool_results:
                        messages.append({
                            "role": "tool",
                            "content": result["result"],
                            "tool_call_id": result["tool_call_id"],
                            "name": result["name"]
                        })

                    # Get final response
                    print("🔄 Generating final response with tool results...")
                    final_response = self.llm.invoke(messages)

                    if hasattr(final_response, 'content'):
                        final_content = final_response.content
                    else:
                        final_content = str(final_response)

                    self.add_to_history("assistant", final_content)
                    return final_content
                else:
                    # No valid tools were executed
                    if hasattr(response, 'content'):
                        self.add_to_history("assistant", response.content)
                        return response.content
                    else:
                        error_msg = "Failed to execute any tools. Please try again."
                        self.add_to_history("assistant", error_msg)
                        return error_msg
            else:
                # No tool calls, just return the response
                if hasattr(response, 'content'):
                    final_content = response.content
                else:
                    final_content = str(response)

                self.add_to_history("assistant", final_content)
                return final_content

        except Exception as e:
            error_msg = f"Error processing message: {str(e)}"
            print(f"❌ {error_msg}")
            # Add error to history
            self.add_to_history("assistant", f"I encountered an error: {str(e)}")
            return f"I'm sorry, I encountered an error while processing your request. Please try again.\n\nError: {str(e)}"

    def get_conversation_summary(self) -> str:
        """Get a summary of the conversation"""
        summary = f"Conversation History ({len(self.conversation_history)} messages):\n"
        for i, msg in enumerate(self.conversation_history[-5:], 1):
            role = msg.get('role', 'unknown')
            content = msg.get('content', '')
            summary += f"{i}. [{role.upper()}] {content[:100]}...\n"
        return summary


# Create a chat function that uses the agent
def chat(user_input: str) -> str:
    """Main chat function that routes requests through the agent"""
    return agent.process_message(user_input)


# Create the agent
agent = SimpleTravelAgent(
    llm=llm,
    tools=[search_flights, book_flight],
    system_prompt=SYSTEM_PROMPT
)

print("✅ Travel Agent initialized with tools:")
for tool_name in agent.tools.keys():
    print(f"   • {tool_name}")

✅ Travel Agent initialized with tools:
   • search_flights
   • book_flight


What happens internally:

1. User message is sent to the LLM
2. LLM decides whether to ask questions or call a tool
3. Tool is executed
4. Final response is generated

## Chat Utility Functions

In [ ]:
# def chat(user_message: str) -> str:
#     """
#     Process a user message and return the agent's response

#     Args:
#         user_message: The user's input message

#     Returns:
#         The agent's response
#     """
#     try:
#         # Run the SimpleTravelAgent
#         response = agent.process_message(user_message)
#         return response

#     except Exception as e:
#         return f"An error occurred: {str(e)}"

## 6. Main execution

In [ ]:
# chat("Hello")

In [ ]:
import sys
from io import StringIO

chat_output = widgets.Output()
text_input = widgets.Text(placeholder='Type your message here...', layout=widgets.Layout(width='auto'))
display(chat_output, text_input)

def on_submit(sender):
    with chat_output:
        user_message = sender.value
        print(f"\n👩‍💻 User: {user_message}")
        sender.value = '' # Clear the input box

        # Redirect stdout to capture agent's internal prints
        old_stdout = sys.stdout
        redirected_output = StringIO()
        sys.stdout = redirected_output

        try:
            response = chat(user_message)
            # Restore stdout
            sys.stdout = old_stdout
            # Print captured output to chat_output
            captured_prints = redirected_output.getvalue().strip()
            if captured_prints:
                print(captured_prints)
            print(f"🤖 Agent: {response}")
        except Exception as e:
            sys.stdout = old_stdout # Ensure stdout is restored even on error
            print(f"An error occurred: {str(e)}")

text_input.on_submit(on_submit)

# Initial greeting from the agent
with chat_output:
    print("🤖 Agent: Hello! I'm your travel agent assistant. How can I help you today?")

Output()

Text(value='', layout=Layout(width='auto'), placeholder='Type your message here...')